In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../')
print(os.getcwd())
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.notebook_utils import fetchIndicators, genDataForAll
from load_config import load_config
from v2.strategy.indicators.momentum import Momentum
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from xgboost import XGBClassifier

/Users/rosscopeland/Desktop/personal/code/vivaldi/back_testing


In [2]:
#adding in indicators
model = Trading(load_config("config.hjson"))
dataset_list = []

for list_d, name in model.df_groups:
    inter_df_list = []
    print(name)
    for d in list_d:
        now = time.time()
        cur_dataset = d
        my_inds = fetchIndicators(['stochastic_oscillator', 'variance','rsi', 'cci', 'ema', 'psar', 'beta', 'natr', 'macd', 'sma', 'smma', 'optimal_v2'])
        genDataForAll(cur_dataset, my_inds)
        mom_ind = Momentum(_params=[Param(2, 1000, 0, 'period', 15)])
        mom_ind.genData(cur_dataset, gen_new_values=False)
        cur_dataset.dropna(inplace=True)
        mm_scalar = MinMaxScaler()
        # cur_dataset[["rsi", "stosc_k", "stosc_d"]] = mm_scalar.fit_transform(cur_dataset[["rsi", "stosc_k", "stosc_d"]])
        inter_df_list.append(cur_dataset)
        print("took {} seconds".format(time.time() - now))
    inter_df_list = pd.concat(inter_df_list)
    mm_scalar = MinMaxScaler()
    inter_df_list[['slowk', 'slowd', 'Variance', 'RSI', 'CCI', 'EMA', 'PSAR', 'Beta', 'NATR',
       'MACD', 'MACD_signal', 'MACD_hist', 'SMA', 'SMMA', 'Momentum']] = mm_scalar.fit_transform(inter_df_list[['slowk', 'slowd', 'Variance', 'RSI', 'CCI', 'EMA', 'PSAR', 'Beta', 'NATR',
       'MACD', 'MACD_signal', 'MACD_hist', 'SMA', 'SMMA', 'Momentum']])
    dataset_list.append(inter_df_list)
final_dataset = pd.concat(dataset_list)

#first try 172.11971712112427

BTCUSDT-1m
took 3.105715036392212 seconds
took 1.2462499141693115 seconds
took 3.3135530948638916 seconds
took 2.796541213989258 seconds
took 3.2370030879974365 seconds
took 1.4775080680847168 seconds
took 2.118920087814331 seconds
took 1.951470136642456 seconds
took 2.0174741744995117 seconds
took 1.6485209465026855 seconds
took 1.452357292175293 seconds
took 3.529942035675049 seconds
ETHUSDT-1m
took 2.746431827545166 seconds
took 1.145374059677124 seconds
took 3.1037020683288574 seconds
took 2.631814956665039 seconds
took 3.2427399158477783 seconds
took 1.4297549724578857 seconds
took 2.1192169189453125 seconds
took 2.017275810241699 seconds
took 1.8947150707244873 seconds
took 1.7187519073486328 seconds
took 1.4568970203399658 seconds
took 3.5595080852508545 seconds
XRPUSDT-1m
took 1.197274923324585 seconds
took 2.5740437507629395 seconds
took 3.2869532108306885 seconds
took 1.6001520156860352 seconds
took 2.2082509994506836 seconds
took 1.9521808624267578 seconds
took 2.01335573196

In [11]:
def filter_optimal_buy(optimal):
    if optimal > 0.0:
        return abs(optimal)
    return 0.0

def filter_optimal_sell(optimal):
    if optimal < 0.0:
        return abs(optimal)
    return 0.0

# for dataset, name in model.dfs:
#     dataset['optimal'] = dataset.apply(lambda x: filter_optimal(x.optimal), axis=1)
final_dataset['optimal_buy'] = final_dataset.apply(lambda x: filter_optimal_buy(x.optimal), axis=1)
final_dataset['optimal_sell'] = final_dataset.apply(lambda x: filter_optimal_sell(x.optimal), axis=1)
final_dataset

,time,high,low,close,open,volume,slowk,slowd,Variance,RSI,...,NATR,MACD,MACD_signal,MACD_hist,SMA,SMMA,optimal,Momentum,optimal_buy,optimal_sell
708,1502985059999,4423.6300,4423.6300,4422.065949,4423.6300,0.321789,0.579117,0.770680,0.002002,0.465860,...,0.059106,0.531440,0.567140,0.554448,0.092017,0.089561,0.000000,0.518795,0.0,0.000000
709,1502985119999,4423.6300,4423.6300,4422.847975,4423.6300,0.127402,0.576326,0.769949,0.002019,0.465860,...,0.055161,0.531251,0.567025,0.554292,0.091993,0.089548,-0.943649,0.523655,0.0,0.943649
710,1502985179999,4412.3800,4412.3800,4417.613987,4412.3800,0.757878,0.573026,0.769196,0.002052,0.443401,...,0.055745,0.531014,0.566909,0.554062,0.091961,0.089527,0.000000,0.524121,0.0,0.000000
711,1502985239999,4412.3800,4377.5400,4397.576994,4412.3800,1.709299,0.568152,0.768416,0.002166,0.382338,...,0.065347,0.530622,0.566790,0.553598,0.091900,0.089484,0.000000,0.518874,0.0,0.000000
712,1502985299999,4410.3500,4371.4500,4384.513497,4377.5400,0.607785,0.563119,0.767608,0.002295,0.372792,...,0.075503,0.530208,0.566669,0.553101,0.091836,0.089437,0.000000,0.516144,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223224,1606715999999,6.5438,6.5438,6.543045,6.5438,7.830000,0.608098,0.744970,0.000320,0.558444,...,0.025388,0.541719,0.563212,0.503694,0.192989,0.193979,-0.524680,0.381182,0.0,0.524680
223225,1606719659999,6.5426,6.5321,6.537573,6.5375,924.510000,0.607436,0.744119,0.000319,0.544192,...,0.027047,0.541460,0.563227,0.503232,0.192990,0.193964,0.000000,0.379928,0.0,0.000000
223226,1606719719999,6.5350,6.5266,6.532086,6.5350,521.450000,0.606430,0.743275,0.000324,0.537667,...,0.027642,0.541181,0.563240,0.502738,0.192985,0.193945,0.000000,0.378370,0.0,0.000000
223227,1606719779999,6.5287,6.5211,6.527093,6.5268,3475.220000,0.605283,0.742438,0.000333,0.532385,...,0.027969,0.540885,0.563250,0.502218,0.192974,0.193923,0.000000,0.377146,0.0,0.000000


In [13]:
dataset = final_dataset.drop(["optimal","volume", "time", "high", "low", "open"], axis=1)
dataset

,close,slowk,slowd,Variance,RSI,CCI,EMA,PSAR,Beta,NATR,MACD,MACD_signal,MACD_hist,SMA,SMMA,Momentum,optimal_buy,optimal_sell
708,4422.065949,0.579117,0.770680,0.002002,0.465860,0.489886,0.090894,0.555061,0.416967,0.059106,0.531440,0.567140,0.554448,0.092017,0.089561,0.518795,0.0,0.000000
709,4422.847975,0.576326,0.769949,0.002019,0.465860,0.490330,0.090874,0.555061,0.416905,0.055161,0.531251,0.567025,0.554292,0.091993,0.089548,0.523655,0.0,0.943649
710,4417.613987,0.573026,0.769196,0.002052,0.443401,0.484324,0.090840,0.555433,0.416405,0.055745,0.531014,0.566909,0.554062,0.091961,0.089527,0.524121,0.0,0.000000
711,4397.576994,0.568152,0.768416,0.002166,0.382338,0.471841,0.090761,0.556585,0.412223,0.065347,0.530622,0.566790,0.553598,0.091900,0.089484,0.518874,0.0,0.000000
712,4384.513497,0.563119,0.767608,0.002295,0.372792,0.470264,0.090676,0.556786,0.412168,0.075503,0.530208,0.566669,0.553101,0.091836,0.089437,0.516144,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223224,6.543045,0.608098,0.744970,0.000320,0.558444,0.473253,0.194032,0.538971,0.514761,0.025388,0.541719,0.563212,0.503694,0.192989,0.193979,0.381182,0.0,0.524680
223225,6.537573,0.607436,0.744119,0.000319,0.544192,0.465436,0.193994,0.539515,0.519181,0.027047,0.541460,0.563227,0.503232,0.192990,0.193964,0.379928,0.0,0.000000
223226,6.532086,0.606430,0.743275,0.000324,0.537667,0.460534,0.193948,0.539770,0.516984,0.027642,0.541181,0.563240,0.502738,0.192985,0.193945,0.378370,0.0,0.000000
223227,6.527093,0.605283,0.742438,0.000333,0.532385,0.457032,0.193898,0.539979,0.513520,0.027969,0.540885,0.563250,0.502218,0.192974,0.193923,0.377146,0.0,0.000000


In [14]:
train, test = train_test_split(dataset, test_size=0.2, shuffle=False)

In [15]:
train_X = train.drop(["optimal_buy", "optimal_sell", "close"], axis=1)

buy_train_y= train[["optimal_buy"]]
sell_train_y = train[["optimal_sell"]]

test_X = test.drop(["optimal_buy", "optimal_sell", "close"], axis=1)

buy_test_y= test[["optimal_buy"]]
sell_test_y = test[["optimal_sell"]]

In [16]:
#making more simple classifer as a baseline
clf_buy = RandomForestRegressor(random_state=69420, n_jobs=-1, min_samples_leaf=2)#, n_estimators=10)

clf_buy.fit(train_X.values, buy_train_y.values)

print("Score of classifer is {} ".format(clf_buy.score(test_X.values, buy_test_y.values)))

for i, v in enumerate(clf_buy.feature_importances_):
    print('i: {}, Feature: {}, Score: {}'.format(i, balanced_data_X.columns[i], clf_buy.feature_importances_[i]))

KeyboardInterrupt: 

In [ ]:
#making more simple classifer as a baseline
clf_sell = RandomForestRegressor(random_state=69420, n_jobs=-1, min_samples_leaf=2)#, n_estimators=10)

clf_sell.fit(train_X.values, sell_train_y.values)

print("Score of classifer is {} ".format(clf_sell.score(test_X.values, sell_test_y.values)))

for i, v in enumerate(clf_sell.feature_importances_):
    print('i: {}, Feature: {}, Score: {}'.format(i, balanced_data_X.columns[i], clf_sell.feature_importances_[i]))

In [ ]:
def insert_buys(row):
    if row.predict_buy > 0.5:
        return row.close
    # if row.predict == 2.0 :# and heat_val > 0.6:
    #     return row.close
    else:
        return None

def insert_sells(row):
    if row.predict_sell > 0.5:
        return row.close
    # if row.predict == 0.0:
    #     return row.close
    else:
        return None

In [ ]:
test_graph = test.drop(["optimal_sell", "optimal_buy"], axis=1)
test_graph.dropna(inplace=True)
test_graph

In [ ]:
test_graph["predict_sell"] = clf_sell.predict_proba(test_X.values)[:,1]
test_graph["predict_buy"] = clf_buy.predict_proba(test_X.values)[:,1]

test_graph["sell"] = test_graph.apply(lambda x: insert_sells(x), axis=1)
test_graph["buy"] = test_graph.apply(lambda x: insert_buys(x), axis=1)

In [ ]:
first_amnt = test_graph[3000:5000]
first_amnt.reset_index(inplace=True, drop=True)

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
plt.figure(figsize=(20,10))

# axes = plt.gca()
# axes.set_ylim([0,10000])


plt.scatter(x=first_amnt.index, y=first_amnt['buy'], color='green')
plt.scatter(x=first_amnt.index, y=first_amnt['sell'], color='red')

plt.plot(first_amnt.index, first_amnt['close'], color='blue')
plt.show()